In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import numpy as np
import os
import math
from matplotlib.animation import FuncAnimation
from matplotlib.patches import Rectangle, FancyArrowPatch
import seaborn as sns

In [2]:
session_df = pd.read_csv("D:\FOVmeasure\Measures\measures\model_sessionplot_df.csv")
Asession_df = pd.read_csv("D:\FOVmeasure\Measures\measures\model_Asessionplot_df.csv")
trial_df = pd.read_csv("D:\FOVmeasure\Measures\measures\model_trial_df.csv")

Psession_df = pd.read_csv("D:\FOVmeasure\Measures\Pdata\Psessionplot.csv")
Ptrial_df = pd.read_csv("D:\FOVmeasure\Measures\Pdata\Ptrial.csv")

In [3]:
#input some set up
#target point(-2.85,0.108).position point(-2.85,-0.623),width0.71,height1.256,and the rader postion0.45,
# Define the target point
target_x = -2.025
rader_x = 0.45
target_y = 0

 # Define the vertices of the hexagon in (x,y) coordinate pairs
vertices = [(-2.5, -1.37), (-2.5, 1.486), (-0.04,  1.486), (-0.04, 1.123), (0.8465, 1.123), (0.8465, -1.37)]

# Define the target area position, width, and height.target point(-2.85,0.108).position point(-2.85,-0.623),width0.71,height1.256,and the rader postion error 0.32
rect_pos = (-2.2,-0.314 )
rect_width = 0.35
rect_height = 0.628

robot_width = 0.4

criterion_dis = 4
criterion_Yr = 0.314

criterion_Yr_2 = 0.628

x_range = (-2.8,1.8)
y_range = (-1.8,1.8)
time_range = (0,180)
Y_range = (0,0.5)

# Define the path to the directory containing the data, the path to store the output 

diskaddress = "D:/FOVmeasure/"

Measures_dir = os.path.join(diskaddress,"Measures/")
data_dir = os.path.join(Measures_dir,"data/")
Pdata_dir = os.path.join(Measures_dir,"Pdata/")
plot_dir = os.path.join(Measures_dir,"plot/")
plotA_dir = os.path.join(Measures_dir,"plot/Acrossfish")

Baselinedata_dir = os.path.join(diskaddress,"Rawdata/Baselinedata/")
Rotationdata_dir = os.path.join(diskaddress,"Rawdata/Rotationdata/")
Washoutdata_dir = os.path.join(diskaddress,"Rawdata/Washoutdata/")
ReRotationdata_dir = os.path.join(diskaddress,"Rawdata/ReRotationdata/")
ReWashoutdata_dir = os.path.join(diskaddress,"Rawdata/ReWashoutdata/")
fishmeasures_dir = os.path.join(Measures_dir,"measures/")


# Create the directory if it does not exist
if not os.path.exists(Measures_dir):
    os.makedirs(Measures_dir)
if not os.path.exists(Pdata_dir):
    os.makedirs(Pdata_dir)
# Create the directory if it does not exist
if not os.path.exists(fishmeasures_dir):
    os.makedirs(fishmeasures_dir)
    # Create the directory if it does not exist
if not os.path.exists(plot_dir):
    os.makedirs(plot_dir)
if not os.path.exists(plotA_dir):
    os.makedirs(plotA_dir)
# Define a list of stages in the correct order
stages = ['Baseline', 'Rotation', 'Washout', 'ReRotation','ReWashout']
stages_BWR= ['Baseline', 'Rotation', 'Washout']
stages_RReR= ['Rotation', 'ReRotation']
stages_ReRReW= [ 'ReRotation', 'ReWashout']
stages_RW = ['Rotation', 'Washout']


# Define color mapping for trial phases
phase_colors = {
    'Baseline': '#F0E442',  # Yellow
    'Rotation': '#CC79A7',   # Reddish Purple
    'Washout': '#F0E442',  # Yellow
    'ReRotation': '#56B4E9',  # Sky Blue
    'ReWashout':   '#E69F00',  # Orange
}
phase_colors_BWR = {
    'Baseline': '#F0E442',  # Yellow
    'Rotation': '#CC79A7',   # Reddish Purple
    'Washout': '#F0E442',  # Yellow
}




In [4]:
def plot_sesssionmeasure_by_stage(sessiondf,sessionmeasure,  y_label, ylim=(-10, 10)):
    """
    Plots the specified measure for each fish by stage.

    Parameters:
        df (DataFrame): The combined DataFrame containing the data.
        measure (str): The column name of the measure to plot.
        y_label (str): The label for the y-axis.
        ylim (tuple): The limits for the y-axis.
    """
    for fish in sessiondf['fish_id'].unique():
        plt.figure(figsize=(14, 4.8))  # Set figure size
        
        for stage in stages:
            # Filter data for the current fish and stage

            sessiondata_filter = sessiondf[(sessiondf['fish_id'] == fish) & (sessiondf['session'] == stage)]
            # Plot the data with the appropriate color for the stage
            plt.plot(sessiondata_filter['plotday'], sessiondata_filter[sessionmeasure], color=phase_colors[stage], label=stage)


        plt.xlabel('Plot Day')  # Set x-axis label
        plt.ylabel(y_label)  # Set y-axis label
        plt.title(f'Fish {fish} - {y_label} by Stage')  # Set plot title
      #  plt.legend(title='Stage', bbox_to_anchor=(1, 1))  # Configure legend
        plt.ylim(ylim)  # Set fixed data range for y-axis
        plt.grid(False)  # Show grid
        # Define the output directory and file path
        output_file_path = os.path.join(plot_dir, f'{fish}')
        
        # Create the directory if it does not exist
        if not os.path.exists(output_file_path):
            os.makedirs(output_file_path)
        
        # Save the plot to a file
        plt.savefig(os.path.join(output_file_path,f'{fish}_{sessionmeasure}_smplot.png'), bbox_inches='tight')
        
        plt.close()  # Close the plot to free memory

In [12]:
def plot_sessionmeasure_relation(sessiondf, sessionmeasure1, sessionmeasure2, y_label, ylim=(-10, 10)):
    """
    Plots the specified measure for each fish by stage.

    Parameters:
        sessiondf (DataFrame): The combined DataFrame containing the data.
        sessionmeasure1 (str): The column name of the measure to plot on the x-axis.
        sessionmeasure2 (str): The column name of the measure to plot on the y-axis.
        y_label (str): The label for the y-axis.
        ylim (tuple): The limits for the y-axis.
    """
    for fish in sessiondf['fish_id'].unique():
        for stage in sessiondf['session'].unique():
            plt.figure(figsize=(14, 4.8))  # Set figure size
            # Filter data for the current fish and stage
            sessiondata_filter = sessiondf[(sessiondf['fish_id'] == fish) & (sessiondf['session'] == stage)]
            
            # Create a scatter plot with color based on 'plotday'
            scatter = plt.scatter(sessiondata_filter[sessionmeasure1], 
                                  sessiondata_filter[sessionmeasure2], 
                                  c=sessiondata_filter['plotday'], 
                                  cmap='viridis',  # You can choose any colormap you like
                                  edgecolor='k')  # Optional: add edge color to points for better visibility

            plt.xlabel(sessionmeasure1)  # Set x-axis label
            plt.ylabel(sessionmeasure2)  # Set y-axis label
            plt.title(f'Fish {fish} - {y_label}')  # Set plot title
            plt.ylim(ylim)  # Set fixed data range for y-axis
            plt.grid(False)  # Show grid

            # Add colorbar to show the mapping of colors to 'plotday'
            cbar = plt.colorbar(scatter)
            cbar.set_label('Plot Day')

            # Define the output directory and file path
            output_file_path = os.path.join('plot_dir', f'{fish}', f'{stage}')
            
            # Create the directory if it does not exist
            if not os.path.exists(output_file_path):
                os.makedirs(output_file_path)
            
            # Save the plot to a file
            plt.savefig(os.path.join(output_file_path, f'{fish}_{sessionmeasure1}_{sessionmeasure2}.png'), bbox_inches='tight')
            plt.close()  # Close the plot to free memor

In [4]:
def plot_measure_by_stage(sessiondf,trialdf,sessionmeasure, trialmeasure, y_label, ylim=(-10, 10)):
    """
    Plots the specified measure for each fish by stage.

    Parameters:
        df (DataFrame): The combined DataFrame containing the data.
        measure (str): The column name of the measure to plot.
        y_label (str): The label for the y-axis.
        ylim (tuple): The limits for the y-axis.
    """
    for fish in sessiondf['fish_id'].unique():
        plt.figure(figsize=(14, 4.8))  # Set figure size
        
        for stage in stages:
            # Filter data for the current fish and stage
            trialdata_filter = trialdf[(trialdf['fish_id'] == fish) & (trialdf['session'] == stage)]
            sessiondata_filter = sessiondf[(sessiondf['fish_id'] == fish) & (sessiondf['session'] == stage)]
            # Plot the data with the appropriate color for the stage
            plt.plot(sessiondata_filter['plotday'], sessiondata_filter[sessionmeasure], color=phase_colors[stage], label=stage)
            plt.scatter(trialdata_filter['plotday'], trialdata_filter[trialmeasure], color=phase_colors[stage], s=1, label=stage)

        plt.xlabel('Plot Day')  # Set x-axis label
        plt.ylabel(y_label)  # Set y-axis label
        plt.title(f'Fish {fish} - {y_label} by Stage')  # Set plot title
       # plt.legend(title='Stage', bbox_to_anchor=(1, 1))  # Configure legend
        plt.ylim(ylim)  # Set fixed data range for y-axis
        plt.grid(False)  # Show grid
        # Define the output directory and file path
        output_file_path = os.path.join(plot_dir, f'{fish}')
        
        # Create the directory if it does not exist
        if not os.path.exists(output_file_path):
            os.makedirs(output_file_path)
        
        # Save the plot to a file
        plt.savefig(os.path.join(output_file_path,f'{fish}_{sessionmeasure}_plot.png'), bbox_inches='tight')
        
        plt.close()  # Close the plot to free memory

In [7]:
def plot_mutimeasure_by_stage(sessiondf,trialdf,sessionmeasure1,sessionmeasure2,sessionmeasure3,sessionmeasure4, trialmeasure, y_label, ylim=(-10, 10)):
    """
    Plots the specified measure for each fish by stage.

    Parameters:
        df (DataFrame): The combined DataFrame containing the data.
        measure (str): The column name of the measure to plot.
        y_label (str): The label for the y-axis.
        ylim (tuple): The limits for the y-axis.
    """
    for fish in sessiondf['fish_id'].unique():
        plt.figure(figsize=(14, 4.8))  # Set figure size
        
        for stage in stages:
            # Filter data for the current fish and stage
            trialdata_filter = trialdf[(trialdf['fish_id'] == fish) & (trialdf['session'] == stage)]
            sessiondata_filter = sessiondf[(sessiondf['fish_id'] == fish) & (sessiondf['session'] == stage)]
            # Plot the data with the appropriate color for the stage
            plt.plot(sessiondata_filter['plotday'], sessiondata_filter[sessionmeasure1], color=phase_colors[stage], label=stage)
            plt.plot(sessiondata_filter['plotday'], sessiondata_filter[sessionmeasure2], color=phase_colors[stage], label=stage)
            plt.plot(sessiondata_filter['plotday'], sessiondata_filter[sessionmeasure3], color=phase_colors[stage], label=stage)
            plt.plot(sessiondata_filter['plotday'], sessiondata_filter[sessionmeasure4], color=phase_colors[stage], label=stage)
            plt.scatter(trialdata_filter['plotday'], trialdata_filter[trialmeasure], color=phase_colors[stage], s=1, label=stage)

        plt.xlabel('Plot Day')  # Set x-axis label
        plt.ylabel(y_label)  # Set y-axis label
        plt.title(f'Fish {fish} - {y_label} by Stage')  # Set plot title
      #  plt.legend(title='Stage', bbox_to_anchor=(1, 1))  # Configure legend
        plt.ylim(ylim)  # Set fixed data range for y-axis
        plt.grid(False)  # Show grid
        # Define the output directory and file path
        output_file_path = os.path.join(plot_dir, f'{fish}')
        
        # Create the directory if it does not exist
        if not os.path.exists(output_file_path):
            os.makedirs(output_file_path)
        
        # Save the plot to a file
        plt.savefig(os.path.join(output_file_path,f'{fish}_muti{trialmeasure}_plot.png'), bbox_inches='tight')
        
        plt.close()  # Close the plot to free memory

In [8]:
plot_mutimeasure_by_stage(session_df,trial_df,'meanYds','medianYds','moving_mean_Yds','moving_median_Yds', 'Yds', 'mutimeasure of Yds', ylim=(-10, 10))

In [51]:
plot_mutimeasure_by_stage(session_df,trial_df,'meandistance','mediandistance','moving_mean_ds','moving_median_ds', 'ds', 'mutimeasure of distance', ylim=(0, 25))

In [52]:
plot_mutimeasure_by_stage(session_df,trial_df,'meandus','mediandus','moving_mean_dus','moving_median_dus', 'dus', 'mutimeasure of dus', ylim=(-300, 400))

In [53]:
plot_mutimeasure_by_stage(session_df,trial_df,'meanduds','medianduds','moving_mean_duds','moving_median_duds', 'duds', 'mutimeasure of duds', ylim=(-0.02, 0.08))

In [54]:
plot_mutimeasure_by_stage(session_df,trial_df,'meanEdis','medianEdis','moving_mean_Edis','moving_median_Edis', 'Edis', 'mutimeasure of Edis', ylim=(0, 3))

In [55]:
plot_mutimeasure_by_stage(session_df,trial_df,'meanEAng','medianEAng','moving_mean_EAng','moving_median_EAng', 'EAng', 'mutimeasure of EAng', ylim=(-3.14, 3.14))

In [56]:
plot_sesssionmeasure_by_stage(session_df,'stdEdis', 'std of end distance to target', ylim=(0, 2))


In [57]:
plot_sesssionmeasure_by_stage(session_df,'stdEAng', 'std of end Angle to target', ylim=(0, 3.5))

In [58]:
plot_sesssionmeasure_by_stage(session_df,'circular_mean_radians', 'circular mean of end Angle to target', ylim=(-3.14,3.14))

In [59]:
plot_sesssionmeasure_by_stage(session_df,'EST', 'success times of one session', ylim=(0, 6))

In [60]:
plot_measure_by_stage(session_df,trial_df, 'medianYds','Yds','Median Yds')
plot_measure_by_stage(session_df,trial_df, 'meanYds','Yds','Mean Yds')

In [20]:
plot_measure_by_stage(session_df,trial_df, 'medianYdsds','Ydsds','Median Ydsds', ylim=(-20, 20))

In [61]:
plot_measure_by_stage(session_df,trial_df, 'moving_median_Yds','Yds','Moving Median Yds')
plot_measure_by_stage(session_df,trial_df, 'moving_mean_Yds','Yds','Moving Mean Yds')


In [62]:
plot_measure_by_stage(session_df,trial_df, 'mediandistance','ds','Median ditances', ylim=(0, 25))
plot_measure_by_stage(session_df,trial_df, 'meandistance','ds','Mean distance', ylim=(0, 25))
plot_measure_by_stage(session_df,trial_df, 'moving_median_ds','ds','Moving Median distance', ylim=(0, 25))
plot_measure_by_stage(session_df,trial_df, 'moving_mean_ds','ds','Moving Mean distance', ylim=(0, 25))


In [63]:
plot_measure_by_stage(session_df,trial_df, 'mediandus','dus','Median correct direction ratio', ylim=(-300, 400))
plot_measure_by_stage(session_df,trial_df, 'meandus','dus','Mean correct direction ratio', ylim=(-300, 400))
plot_measure_by_stage(session_df,trial_df, 'moving_median_dus','dus','Moving Median correct direction ratio', ylim=(-300, 400))
plot_measure_by_stage(session_df,trial_df, 'moving_mean_dus','dus','Moving Mean correct direction ratio', ylim=(-300, 400))

In [64]:
plot_measure_by_stage(session_df,trial_df, 'medianduds','duds','Median correct direction ratio2', ylim=(-0.02, 0.08))
plot_measure_by_stage(session_df,trial_df, 'meanduds','duds','Mean correct direction ratio2', ylim=(-0.02, 0.08))
plot_measure_by_stage(session_df,trial_df, 'moving_median_duds','duds','Moving Median correct direction ratio2', ylim=(-0.02, 0.08))
plot_measure_by_stage(session_df,trial_df, 'moving_mean_duds','duds','Moving Mean correct direction ratio2', ylim=(-0.02, 0.08))

In [65]:
plot_measure_by_stage(session_df,trial_df, 'medianEdis','Edis','Median end ditance to target', ylim=(0, 3))
plot_measure_by_stage(session_df,trial_df, 'meanEdis','Edis','Mean end ditance to target', ylim=(0, 3))
plot_measure_by_stage(session_df,trial_df, 'moving_median_Edis','Edis','Moving Median end ditance to target', ylim=(0, 3))
plot_measure_by_stage(session_df,trial_df, 'moving_mean_Edis','Edis','Moving Mean end ditance to target', ylim=(0, 3))


In [66]:
plot_measure_by_stage(session_df,trial_df, 'medianEAng','EAng','Median end Angle to target', ylim=(-3.14, 3.14))
plot_measure_by_stage(session_df,trial_df, 'meanEAng','EAng','Mean end Angle to target', ylim=(-3.14, 3.14))
plot_measure_by_stage(session_df,trial_df, 'moving_median_EAng','EAng','Moving Median end Angle to target', ylim=(-3.14, 3.14))
plot_measure_by_stage(session_df,trial_df, 'moving_mean_EAng','EAng','Moving Mean end Angle to target', ylim=(-3.14, 3.14))

In [67]:
plot_sesssionmeasure_by_stage(session_df,'σP', 'std of end point', ylim=(0, 2))

In [12]:
plot_measure_by_stage(Psession_df,Ptrial_df, 'medianEDire','EDire','Median direction from start to end', ylim=(-40, 40))
plot_measure_by_stage(Psession_df,Ptrial_df, 'meanEDire','EDire','Mean direction from start to end', ylim=(-40, 40))

In [14]:
plot_measure_by_stage(Psession_df,Ptrial_df, 'circular_mean_degrees','EDire','circular_mean direction from start to end', ylim=(-40, 40))

In [13]:
plot_sessionmeasure_relation(Psession_df,'Fdis','EDire','The relation of EDir and Fdis')

In [5]:
plot_measure_by_stage(Psession_df,Ptrial_df, 'medianFCdis','Fcompletedis','Median Fancy complete trajctory length', ylim=(0, 15))